Health and Exercise Tracker Analysis Notebook

The following cell pulls the live tracker from Google Drive so I don't have to redownaload the file to a data folder every time.

In [1]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import pandas as pd
import requests
from io import BytesIO
from datetime import datetime

spreadsheetId = "16EZhzrGxpV86c_1Axe9RpDaQNBbIbFWPVzkUyhOMIiA"  # <--- Please set the Spreadsheet ID.

# 1. Download the Google Spreadsheet as XLSX format.
gauth = GoogleAuth()
gauth.LocalWebserverAuth()
url = "https://www.googleapis.com/drive/v3/files/" + spreadsheetId + "/export?mimeType=application%2Fvnd.openxmlformats-officedocument.spreadsheetml.sheet"
res = requests.get(url, headers={"Authorization": "Bearer " + gauth.attr['credentials'].access_token})

# 2. The downloaded XLSX data is read with `pd.read_excel`.
sheet = "Sheet1"
df = pd.read_excel(BytesIO(res.content), usecols=None, sheet_name=sheet)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=895221966072-ivfclv15clemuid8o8fphc3205ccooh2.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


Here is a preview of the DataFrame.

In [2]:
df.head()

,date,weight_kg,hours_slept,sleep_quality,bpm,steps,walk_distance_km,walk_time,run_distance_km,run_time,cycle_distance_km,cycle_time,weights_intensity,weights_time,notes
0,07.03.2025,77.0,06:30:00,4,54,18721.0,NaN,NaN,5.10,30.32,NaN,NaN,NaN,NaN,NaN
1,08.03.2025,75.0,10:36:00,7,56,7194.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,09.03.2025,76.2,07:07:00,5,58,8512.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN
3,10.03.2025,77.5,08:32:00,6,55,14294.0,3.74,0.41,3.77,0.38,NaN,NaN,NaN,NaN,"St Ramon (Uphill), sprained ankle, slow walk back"
4,11.03.2025,76.6,08:41:00,5,54,12825.0,NaN,NaN,NaN,NaN,19.69,0.56,NaN,NaN,NaN


For the moment we will convert all the NaN's to a 0 to make the data organisation clear, we may have to revisit this later.

In [3]:
# df = df.fillna(0)

Decided against using a fillna() function as it will skew future calculations based on the 0's.

And here we can see the data types for the individual columns.

In [4]:
df.dtypes

date                  object
weight_kg            float64
hours_slept           object
sleep_quality          int64
bpm                    int64
steps                float64
walk_distance_km     float64
walk_time            float64
run_distance_km      float64
run_time             float64
cycle_distance_km    float64
cycle_time           float64
weights_intensity    float64
weights_time         float64
notes                 object
dtype: object

So here we can see that we have some data types that will make life difficult for us in the future.

We could just change the data types in the original google sheets document but that wouldn't be good practice.

Let's start by trying to see if we can turn these into universal values.

First we'll convert the date into a date format:

In [5]:
df["date"] = pd.to_datetime(df["date"], format='mixed')
df["date"] = df['date'].dt.strftime('%m/%d/%Y')

In [6]:
df["date"]

0     07/03/2025
1     08/03/2025
2     09/03/2025
3     10/03/2025
4     11/03/2025
5     12/03/2025
6     03/13/2025
7     03/14/2025
8     03/15/2025
9     03/16/2025
10    03/17/2025
11    03/18/2025
12    03/19/2025
13    03/20/2025
14    03/21/2025
15    03/22/2025
16    03/23/2025
17    03/24/2025
Name: date, dtype: object

Now we can see that that the first column has the correct format:

In [7]:
df.dtypes

date                  object
weight_kg            float64
hours_slept           object
sleep_quality          int64
bpm                    int64
steps                float64
walk_distance_km     float64
walk_time            float64
run_distance_km      float64
run_time             float64
cycle_distance_km    float64
cycle_time           float64
weights_intensity    float64
weights_time         float64
notes                 object
dtype: object

We can set the individual columns according to their starting dtypes, see the column ["weight"] here:

In [8]:
df["weight_kg"] = df['weight_kg'].astype(float)

In [9]:
df.dtypes

date                  object
weight_kg            float64
hours_slept           object
sleep_quality          int64
bpm                    int64
steps                float64
walk_distance_km     float64
walk_time            float64
run_distance_km      float64
run_time             float64
cycle_distance_km    float64
cycle_time           float64
weights_intensity    float64
weights_time         float64
notes                 object
dtype: object

In [10]:
df.head()

,date,weight_kg,hours_slept,sleep_quality,bpm,steps,walk_distance_km,walk_time,run_distance_km,run_time,cycle_distance_km,cycle_time,weights_intensity,weights_time,notes
0,07/03/2025,77.0,06:30:00,4,54,18721.0,NaN,NaN,5.10,30.32,NaN,NaN,NaN,NaN,NaN
1,08/03/2025,75.0,10:36:00,7,56,7194.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,09/03/2025,76.2,07:07:00,5,58,8512.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN
3,10/03/2025,77.5,08:32:00,6,55,14294.0,3.74,0.41,3.77,0.38,NaN,NaN,NaN,NaN,"St Ramon (Uphill), sprained ankle, slow walk back"
4,11/03/2025,76.6,08:41:00,5,54,12825.0,NaN,NaN,NaN,NaN,19.69,0.56,NaN,NaN,NaN


Now we'll look at something a little more complicated, converting the object type "hours_slept" column into am HH:MM format.

In [11]:
df["hours_slept"] = df["hours_slept"].apply(lambda x: x[:-3])

TypeError: 'datetime.time' object is not subscriptable

In [ ]:
df["hours_slept"] = pd.to_datetime(df["hours_slept"], format='%H:%M:%S', errors='coerce', utc = True).dt.time

In [ ]:
df.head()

,date,weight_kg,hours_slept,sleep_quality,bpm,steps,walk_distance_km,walk_time,run_distance_km,run_time,cycle_distance_km,cycle_time,weights_intensity,weights_time,notes
0,07/03/2025,77.0,06:30:00,4,54,18721.0,NaN,NaN,5.10,30.32,NaN,NaN,NaN,NaN,NaN
1,08/03/2025,75.0,10:36:00,7,56,7194.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,09/03/2025,76.2,07:07:00,5,58,8512.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN
3,10/03/2025,77.5,08:32:00,6,55,14294.0,3.74,0.41,3.77,0.38,NaN,NaN,NaN,NaN,"St Ramon (Uphill), sprained ankle, slow walk back"
4,11/03/2025,76.6,08:41:00,5,54,12825.0,NaN,NaN,NaN,NaN,19.69,0.56,NaN,NaN,NaN


In [ ]:
df.dtypes

date                  object
weight_kg            float64
hours_slept           object
sleep_quality          int64
bpm                    int64
steps                float64
walk_distance_km     float64
walk_time            float64
run_distance_km      float64
run_time             float64
cycle_distance_km    float64
cycle_time           float64
weights_intensity    float64
weights_time         float64
notes                 object
dtype: object

In [ ]:
df["hours_slept"]

0     06:30:00
1     10:36:00
2     07:07:00
3     08:32:00
4     08:41:00
5     08:06:00
6     07:38:00
7     07:42:00
8     07:01:00
9     07:58:00
10    09:15:00
11    09:11:00
12    07:52:00
13    07:32:00
14    06:51:00
15    08:23:00
16    08:19:00
17    09:42:00
Name: hours_slept, dtype: object

Here we'll look at converting two types simultaneously as we need both as integer values.

In [ ]:
df[["sleep_quality", "bpm"]] = df[["sleep_quality", "bpm"]].astype(int)

In [ ]:
df.dtypes

date                  object
weight_kg            float64
hours_slept           object
sleep_quality          int64
bpm                    int64
steps                float64
walk_distance_km     float64
walk_time            float64
run_distance_km      float64
run_time             float64
cycle_distance_km    float64
cycle_time           float64
weights_intensity    float64
weights_time         float64
notes                 object
dtype: object

In [ ]:
df.head(10)

,date,weight_kg,hours_slept,sleep_quality,bpm,steps,walk_distance_km,walk_time,run_distance_km,run_time,cycle_distance_km,cycle_time,weights_intensity,weights_time,notes
0,07/03/2025,77.0,06:30:00,4,54,18721.0,NaN,NaN,5.10,30.32,NaN,NaN,NaN,NaN,NaN
1,08/03/2025,75.0,10:36:00,7,56,7194.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,09/03/2025,76.2,07:07:00,5,58,8512.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN
3,10/03/2025,77.5,08:32:00,6,55,14294.0,3.74,0.41,3.77,0.38,NaN,NaN,NaN,NaN,"St Ramon (Uphill), sprained ankle, slow walk back"
4,11/03/2025,76.6,08:41:00,5,54,12825.0,NaN,NaN,NaN,NaN,19.69,0.56,NaN,NaN,NaN
5,12/03/2025,75.7,08:06:00,6,56,7474.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,03/13/2025,76.3,07:38:00,4,56,11751.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,03/14/2025,76.3,07:42:00,4,58,16301.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Visit to Stoke
8,03/15/2025,76.3,07:01:00,3,59,20994.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Visit to Stoke
9,03/16/2025,76.3,07:58:00,3,60,8583.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Visit to Stoke


We now have the steps column which requires a simple round to 0 decimal places transformation.

In [ ]:
df["steps"] = (df["steps"]).fillna(0).astype(int)

In [ ]:
df.dtypes

date                  object
weight_kg            float64
hours_slept           object
sleep_quality          int64
bpm                    int64
steps                  int64
walk_distance_km     float64
walk_time            float64
run_distance_km      float64
run_time             float64
cycle_distance_km    float64
cycle_time           float64
weights_intensity    float64
weights_time         float64
notes                 object
dtype: object

In [ ]:
df.head(10)

,date,weight_kg,hours_slept,sleep_quality,bpm,steps,walk_distance_km,walk_time,run_distance_km,run_time,cycle_distance_km,cycle_time,weights_intensity,weights_time,notes
0,07/03/2025,77.0,06:30:00,4,54,18721,NaN,NaN,5.10,30.32,NaN,NaN,NaN,NaN,NaN
1,08/03/2025,75.0,10:36:00,7,56,7194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,09/03/2025,76.2,07:07:00,5,58,8512,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN
3,10/03/2025,77.5,08:32:00,6,55,14294,3.74,0.41,3.77,0.38,NaN,NaN,NaN,NaN,"St Ramon (Uphill), sprained ankle, slow walk back"
4,11/03/2025,76.6,08:41:00,5,54,12825,NaN,NaN,NaN,NaN,19.69,0.56,NaN,NaN,NaN
5,12/03/2025,75.7,08:06:00,6,56,7474,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,03/13/2025,76.3,07:38:00,4,56,11751,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,03/14/2025,76.3,07:42:00,4,58,16301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Visit to Stoke
8,03/15/2025,76.3,07:01:00,3,59,20994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Visit to Stoke
9,03/16/2025,76.3,07:58:00,3,60,8583,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Visit to Stoke


Here is the more efficient and cohesive function to convert the %_time columns to a convenient format.

In [ ]:
def float_to_duration(column: list) -> list:
    for value in column:
      return pd.to_datetime(value, format='%H:%M:%S', errors='coerce', utc=True)

#  .dt.time missing from the end of the function

Here we are passing several values from a variable(list) to the function in the form of a loop.

In [ ]:
columns = ["walk_time", "run_time", "cycle_time", "weights_time"]

for col in columns:
    float_to_duration(df[col])

In [ ]:
df.dtypes

date                  object
weight_kg            float64
hours_slept           object
sleep_quality          int64
bpm                    int64
steps                  int64
walk_distance_km     float64
walk_time            float64
run_distance_km      float64
run_time             float64
cycle_distance_km    float64
cycle_time           float64
weights_intensity    float64
weights_time         float64
notes                 object
dtype: object

In [ ]:
df.head(10)

,date,weight_kg,hours_slept,sleep_quality,bpm,steps,walk_distance_km,walk_time,run_distance_km,run_time,cycle_distance_km,cycle_time,weights_intensity,weights_time,notes
0,07/03/2025,77.0,06:30:00,4,54,18721,NaN,NaN,5.10,30.32,NaN,NaN,NaN,NaN,NaN
1,08/03/2025,75.0,10:36:00,7,56,7194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,09/03/2025,76.2,07:07:00,5,58,8512,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN
3,10/03/2025,77.5,08:32:00,6,55,14294,3.74,0.41,3.77,0.38,NaN,NaN,NaN,NaN,"St Ramon (Uphill), sprained ankle, slow walk back"
4,11/03/2025,76.6,08:41:00,5,54,12825,NaN,NaN,NaN,NaN,19.69,0.56,NaN,NaN,NaN
5,12/03/2025,75.7,08:06:00,6,56,7474,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,03/13/2025,76.3,07:38:00,4,56,11751,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,03/14/2025,76.3,07:42:00,4,58,16301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Visit to Stoke
8,03/15/2025,76.3,07:01:00,3,59,20994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Visit to Stoke
9,03/16/2025,76.3,07:58:00,3,60,8583,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Visit to Stoke


Converting the awkward exercise_time columns to an appropriate time (HH:MM).